In [1]:
from aocd import get_data
import numpy as np

day16 = get_data(day=16,year=2021)

This seems to require a function that can call itself.

In [2]:
def read_packet(bits):
    n = 0
    version = int(bits[n:n+3],base=2)
    versions.append(version)
    typeid = int(bits[n+3:n+6],base=2)
#    print('Version ',version,' Type ID ',typeid)
    n += 6
    if typeid==4:
        literal = ''
        leading = int(bits[n:n+1],base=2)
        literal += bits[n+1:n+5]
        n += 5
        while leading == 1:
            leading = int(bits[n:n+1],base=2)
            literal += bits[n+1:n+5]
            n += 5
#        print('Literal value ',int(literal,base=2),', packet length',n)
        return n
    else:
        lengthtypeid = int(bits[n:n+1],base=2)
        n += 1
        if lengthtypeid == 0:
            sublength = int(bits[n:n+15],base=2)
#            print('Operator with ',sublength,' bits of subpackets')
            n += 15
            l = 0
            while l < sublength:
                l += read_packet(bits[n+l:])
            n += l
        else:
            nsubpackets = int(bits[n:n+11],base=2)
#            print('Operator with ',nsubpackets,' subpackets')
            n += 11                
            for i in range(nsubpackets):
                n += read_packet(bits[n:])
    return n
        

In [3]:
bits = ''
for i in '8A004A801A8002F478':
    bits += bin(int(i,base=16))[2:].zfill(4)

versions = []
read_packet(bits)
print(np.sum(versions))

16


In [4]:
bits = ''
for i in day16:
    bits += bin(int(i,base=16))[2:].zfill(4)


versions = []
read_packet(bits)
print(np.sum(versions))

854


Got it!

In [17]:
def read_packet(bits):
    n = 0
    version = int(bits[n:n+3],base=2)
    typeid = int(bits[n+3:n+6],base=2)
    n += 6
    if typeid==4:
        literal = ''
        leading = int(bits[n:n+1],base=2)
        literal += bits[n+1:n+5]
        n += 5
        while leading == 1:
            leading = int(bits[n:n+1],base=2)
            literal += bits[n+1:n+5]
            n += 5
        ops.append(str(int(literal,base=2)))
        ops.append(',')
        return n
    else:
        if typeid == 0:
            ops.append('np.sum([')
        elif typeid == 1:
            ops.append('np.product([')
        elif typeid == 2:
            ops.append('1*np.min([')
        elif typeid == 3:
            ops.append('1*np.max([')
        elif typeid == 5:
            ops.append('1*np.greater(')
        elif typeid == 6:
            ops.append('1*np.less(')
        elif typeid == 7:
            ops.append('1*np.equal(')
        lengthtypeid = int(bits[n:n+1],base=2)
        n += 1
        if lengthtypeid == 0:
            sublength = int(bits[n:n+15],base=2)
            n += 15
            l = 0
            while l < sublength:
                l += read_packet(bits[n+l:])
                ops.append(',')
            n += l
        else:
            nsubpackets = int(bits[n:n+11],base=2)
            n += 11                
            for i in range(nsubpackets):
                n += read_packet(bits[n:])
                ops.append(',')
        if typeid < 4:
            ops.append(']')
        ops.append(')')
    return n
        

In [18]:
bits = ''
for i in '880086C3E88112':
    bits += bin(int(i,base=16))[2:].zfill(4)

ops = []
read_packet(bits)


55

In [19]:
eval(''.join(ops).replace(',,',',').replace(',]',']').replace(',)',')'))

7

In [20]:
bits = ''
for i in day16:
    bits += bin(int(i,base=16))[2:].zfill(4)

ops = []
read_packet(bits)   
print(eval(''.join(ops).replace(',,',',').replace(',]',']').replace(',)',')')))

181894873364


Not working; try a different approach.

In [22]:
def read_packet(bits):
    n = 0
    version = int(bits[n:n+3],base=2)
    typeid = int(bits[n+3:n+6],base=2)
    n += 6
    if typeid==4:
        literal = ''
        leading = int(bits[n:n+1],base=2)
        literal += bits[n+1:n+5]
        n += 5
        while leading == 1:
            leading = int(bits[n:n+1],base=2)
            literal += bits[n+1:n+5]
            n += 5
        return n,int(literal,base=2)
    else:
        lengthtypeid = int(bits[n:n+1],base=2)
        n += 1
        subvalues = []
        if lengthtypeid == 0:
            sublength = int(bits[n:n+15],base=2)
            n += 15
            l = 0
            while l < sublength:
                m,v = read_packet(bits[n+l:])
                l += m
                subvalues.append(v)
            n += l
        else:
            nsubpackets = int(bits[n:n+11],base=2)
            n += 11                
            for i in range(nsubpackets):
                m,v = read_packet(bits[n:])
                n += m
                subvalues.append(v)
        if typeid == 0:
            return n,np.sum(subvalues)
        elif typeid == 1:
            return n,np.product(subvalues)
        elif typeid == 2:
            return n,np.min(subvalues)
        elif typeid == 3:
            return n,np.max(subvalues)
        elif typeid == 5:
            return n,np.greater(subvalues[0],subvalues[1])*1
        elif typeid == 6:
            return n,np.less(subvalues[0],subvalues[1])*1
        elif typeid == 7:
            return n,np.equal(subvalues[0],subvalues[1])*1
        

In [23]:
bits = ''
for i in '9C0141080250320F1802104A08':
    bits += bin(int(i,base=16))[2:].zfill(4)

read_packet(bits)

(102, 1)

In [24]:
bits = ''
for i in day16:
    bits += bin(int(i,base=16))[2:].zfill(4)

read_packet(bits)

(5332, 181894873364)

I got the exact same answer, so the coding approach wasn't the problem.